In [ ]:
#import sklearn as sk;
import pandas as pd;
import numpy as np; 


Label: Human_Development_Index

Countrys: 
"Name" 
"Region" 

Education: 

Event:

Heath:

Month:

Population:

Quality of Life:

In [ ]:
# load the datasets 
dfCountry = pd.read_csv('./Tables/Country.csv')
dfEducation = pd.read_csv('./Tables/Education.csv')
dfEvent = pd.read_csv('./Tables/Event.csv')
dfFactTable = pd.read_csv('./Tables/Fact_Table.csv')
dfHealth = pd.read_csv('./Tables/Health.csv')
dfMonth = pd.read_csv('./Tables/Month.csv')
dfPopulation = pd.read_csv('./Tables/Population.csv')
dfQualityOfLife = pd.read_csv('./Tables/Quality_of_Life.csv')

dfPopulation = dfPopulation.drop(["PopulationKey", "SurrogateKey", "Country", "Year"], axis=1)
dfEducation = dfEducation.drop(["EducationKey", "SurrogateKey", "Country", "Year"], axis=1)
dfHealth = dfHealth.drop(["HealthKey", "SurrogateKey", "Country", "Year"], axis=1)
dfQualityOfLife = dfQualityOfLife.drop(["QualityofLifeKey", "SurrogateKey"], axis=1)

print(dfEducation.shape[0])
print(dfHealth.shape[0])
print(dfPopulation.shape[0])
print(dfQualityOfLife.shape[0])

canada = dfFactTable.loc[dfFactTable['CountryKey'] == 2]
unitedStates = dfFactTable.loc[dfFactTable['CountryKey'] == 8]
mexico = dfFactTable.loc[dfFactTable['CountryKey'] == 5]
mauritius = dfFactTable.loc[dfFactTable['CountryKey'] == 6]
liberia = dfFactTable.loc[dfFactTable['CountryKey'] == 4]
congo = dfFactTable.loc[dfFactTable['CountryKey'] == 3]
vietnam = dfFactTable.loc[dfFactTable['CountryKey'] == 9]
nepal = dfFactTable.loc[dfFactTable['CountryKey'] == 7]
bangladesh = dfFactTable.loc[dfFactTable['CountryKey'] == 1]

fixedFact = canada.append(unitedStates)
fixedFact = fixedFact.append(mexico)
fixedFact = fixedFact.append(mauritius)
fixedFact = fixedFact.append(liberia)
fixedFact = fixedFact.append(congo)
fixedFact = fixedFact.append(vietnam)
fixedFact = fixedFact.append(nepal)
fixedFact = fixedFact.append(bangladesh)
fixedFact = fixedFact.drop(["MonthKey","CountryKey", "EducationKey",  "QualityofLifeKey", "HealthKey", "EventKey", "PopulationKey", "Development_Index"], axis=1) 

df = pd.concat([dfQualityOfLife, dfHealth, dfPopulation, dfEducation, fixedFact], axis=1)

# print the shape of the dataframes 
print(f"Test Data shape: \n{dfCountry}\n")
print(f"Test Data shape: \n{dfEducation.shape}\n")
print(f"Test Data shape: \n{dfEvent.shape}\n")
print(f"Test Data shape: \n{dfFactTable.shape}\n")
print(f"Test Data shape: \n{dfHealth.shape}\n")
print(f"Test Data shape: \n{dfMonth.shape}\n")
print(f"Test Data shape: \n{dfPopulation.shape}\n")
print(f"Test Data shape: \n{dfQualityOfLife.shape}\n")
print("_____________________________________________\n")


    #scatterPlot


#print(f"Health Scatterplot: \n{scatterHealth}\n")




for item in (df.columns.values):
     scatterPlot = df.plot.scatter(x=item, y='Human_Development_Index', c='orange')
     scatterPlot.xaxis.label.set_color('white')        #setting up X-axis label color to yellow
     scatterPlot.yaxis.label.set_color('white')          #setting up Y-axis label color to blue
     scatterPlot.tick_params(axis='x', colors='white')    #setting up X-axis tick color to red
     scatterPlot.tick_params(axis='y', colors='white')  #setting up Y-axis tick color to black
     scatterPlot.spines['left'].set_color('white')        # setting up Y-axis tick color to red
     scatterPlot.spines['top'].set_color('white')         #setting up above X-axis tick color to red

In [ ]:
boxplot = df.boxplot(column = 'Human_Development_Index')
histogram = df.hist(column='Human_Development_Index')

In [ ]:
#Finds the percentage of nulls within each column
percent_missing = df.isnull().sum() * 100 / len(df)

#Takes any columns that have a % of nulls greater than 1/3, and drops them from the dataframe
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True)
print("\n\n Below are the percentages of nulls in each of our possible features:")
print("_____________________________________________________________________\n ")
print(missing_value_df)
df = df.drop(missing_value_df[percent_missing>33.33].index, axis=1)

df["Population,_total"] = df["Population,_total"].astype('float64')
df["Population,_male"] = df["Population,_male"].astype('float64')
df["Population,_female"] = df["Population,_female"].astype('float64')
df["Quality_of_Life"] = df["Quality_of_Life"].astype('float64')


In [101]:
print(f"dataframe datatypes:\n")
print(df.dtypes)
print("\n")
print("_____________________________________________\n")


dataframe datatypes:

Country                                                            object
Year                                                               object
People_practicing_open_defecation(%_of_population)                float64
%_of_population_using_least_basic_sanitation_services             float64
%_of_population_using_at_least_basic_drinking_water_services      float64
Labor_force,_female(%_of_total_labor_force)                       float64
Unemployment,_male(%_of_male_labor_force)                         float64
Unemployment,_female(%_of_female_labor_force)                     float64
Unemployment,_total(%_of_total_labor_force)                       float64
Domestic_government_health_expenditure_per_capita(current US$)    float64
Immunization_HepB3(%_of_one-year-old_children)                    float64
Adults(ages_15+)_and_children(0-14_years)_living_with_HIV         float64
Out-of-pocket_expenditure(%_of _current_health_expenditure)       float64
Mortality_from_C

In [102]:
numerical_df = df.drop(["Country", "Year"], axis=1)
numerical_df



,People_practicing_open_defecation(%_of_population),%_of_population_using_least_basic_sanitation_services,%_of_population_using_at_least_basic_drinking_water_services,"Labor_force,_female(%_of_total_labor_force)","Unemployment,_male(%_of_male_labor_force)","Unemployment,_female(%_of_female_labor_force)","Unemployment,_total(%_of_total_labor_force)",Domestic_government_health_expenditure_per_capita(current US$),Immunization_HepB3(%_of_one-year-old_children),Adults(ages_15+)_and_children(0-14_years)_living_with_HIV,...,"Birth_rate,_crude_(per_1,000_people)",Rural_population_(%_of_total_population),Urban_population_(%_of_total_population),Age_dependency_ratio_(%_of_working-age_population),"Age_dependency_ratio,_young",School_enrollment_primary_male(%gross),School_enrollment_primary_female(%gross),School_enrollment_tertiary(%gross),Quality_of_Life,Human_Development_Index
0,0.000000,99.805190,99.234640,46.419240,7.01,6.46,6.76,2408.0,14.0,43000.0,...,10.600,19.878,80.122,44.495094,25.540874,97.77119,96.93871,NaN,4.0,0.514
1,0.000000,99.750336,99.234620,46.674060,6.52,6.09,6.32,2747.0,14.0,45000.0,...,10.900,19.787,80.213,44.236996,25.098648,99.23444,98.50983,NaN,4.0,0.521
2,0.000000,99.695950,99.235500,46.855960,6.38,5.65,6.04,3062.0,14.0,46000.0,...,11.200,19.604,80.396,44.034775,24.681334,99.20019,98.72111,63.60006,4.0,0.528
3,0.000000,99.641460,99.236080,46.800793,6.57,5.65,6.14,3260.0,28.0,48000.0,...,11.300,19.422,80.578,43.933903,24.312925,98.10118,97.92086,63.76740,4.0,0.531
4,0.000000,99.586845,99.236360,47.032047,9.53,7.01,8.34,3202.0,42.0,49000.0,...,11.300,19.242,80.758,43.976753,24.010157,98.93639,99.08513,63.06577,4.0,0.543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,1.851918,48.225540,97.229614,28.748392,3.13,7.38,4.35,6.0,98.0,NaN,...,18.824,64.917,35.083,51.260017,43.491890,NaN,NaN,17.87436,4.0,0.693
140,1.020961,49.728413,97.353260,30.174154,3.35,6.75,4.37,6.0,98.0,NaN,...,18.501,64.142,35.858,50.091698,42.362762,111.08681,119.05796,18.15146,4.0,0.696
141,0.220963,51.219707,97.472530,30.364622,3.26,6.65,4.29,7.0,98.0,NaN,...,18.184,63.368,36.632,48.952297,41.268757,112.35745,120.75175,20.56507,3.0,0.700
142,0.000000,52.698704,97.587425,30.514652,3.20,6.55,4.22,NaN,98.0,NaN,...,17.868,62.595,37.405,47.917435,40.254790,NaN,NaN,24.01689,3.0,0.704
